In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-8b-instant"    
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [41]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://job-boards.greenhouse.io/notion/jobs/6586329003")
page_data = loader.load().pop().page_content
print(page_data)

Job Application for Corporate Finance, Revenue at NotionBack to jobsCorporate Finance, RevenueSan Francisco, CaliforniaApplyAbout Us:
We're on a mission to make it possible for every person, team, and company to be able to tailor their software to solve any problem and take on any challenge. Computers may be our most powerful tools, but most of us can't build or modify the software we use on them every day. At Notion, we want to change this with focus, design, and craft.
We've been working on this together since 2016, and have customers like OpenAI, Toyota, Figma, Ramp, and thousands more on this journey with us. Today, we're growing fast and excited for new teammates to join us who are the best at what they do. We're passionate about building a company as diverse and creative as the millions of people Notion reaches worldwide.
Notion is an in person company, and currently requires its employees to come to the office for two Anchor Days (Mondays & Thursdays) and requests that employees

In [43]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`,`skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
[
  {
    "role": "Corporate Finance, Revenue",
    "experience": "8+ years experience in FP&A, including prior experience converting operating metrics (e.g., ARR, usage) to GAAP Revenue",
    "skills": [
      "Experience with financial modeling and analytical mindset",
      "Positive attitude with high aptitude to learn new things quickly. Team player eager to further develop and grow with us",
      "Demonstrate intellectual curiosity, problem-solving skills, and hunger to continually up-level current approaches and processes.",
      "Strong work ethic, reliability, and attention to detail - “get it done” attitude"
    ],
    "description": "Our Corporate Finance team serves as a trusted advisor to the company, informing and guiding critical C-level decisions through insightful analyses & thought partnership. We are looking for a bright, driven individual who will support Executive-level understanding of our key financial metrics and trends, dial in our forecasting accurac

In [44]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Corporate Finance, Revenue',
  'experience': '8+ years experience in FP&A, including prior experience converting operating metrics (e.g., ARR, usage) to GAAP Revenue',
  'skills': ['Experience with financial modeling and analytical mindset',
   'Positive attitude with high aptitude to learn new things quickly. Team player eager to further develop and grow with us',
   'Demonstrate intellectual curiosity, problem-solving skills, and hunger to continually up-level current approaches and processes.',
   'Strong work ethic, reliability, and attention to detail - “get it done” attitude'],
  'description': 'Our Corporate Finance team serves as a trusted advisor to the company, informing and guiding critical C-level decisions through insightful analyses & thought partnership. We are looking for a bright, driven individual who will support Executive-level understanding of our key financial metrics and trends, dial in our forecasting accuracy, and take on high-profile strategic proje

In [45]:
type(json_res)

list

In [46]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [47]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [49]:
links = collection.query(
    query_texts=["Experience in python", "Expertise in React"], 
    n_results=2
).get('metadatas') 
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [50]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [51]:
job

{'role': 'ACG, FW Materials Developer II',
 'experience': 'Not specified',
 'skills': 'Not specified',
 'description': 'Product Development GroupView Job'}

In [53]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Megha SK, a business development executive at Abc. Abc is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Abc 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Abc's portfolio: {link_list}
        Remember you are Megha SK, BDE at Abc. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Efficient Product Development with Abc

Dear [Client Name],

I came across the job description for an ACG, FW Materials Developer II at [Company Name] and was impressed by the Product Development Group's focus on innovation. As a Business Development Executive at Abc, an AI & Software Consulting company, I'd like to introduce you to our expertise in facilitating seamless integration of business processes through automated tools.

At Abc, we've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has a proven track record of delivering high-quality projects that meet and exceed client expectations.

I'd like to highlight a few examples from our portfolio that demonstrate our capabilities:

- **Kotlin Backend Development**: We've successfully developed scalable and efficient backend systems for clients using Kotlin. You can view our portfolio here: https: